# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load and read csv file
city_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
city_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Date
0,0,belushya guba,-0.7393,-90.3518,72.55,31.0,91.0,4.00,EC,1.623310e+09
1,1,puerto ayora,-33.6500,115.3333,62.10,71.0,80.0,16.46,AU,1.623310e+09
2,2,busselton,50.6789,156.1250,47.84,95.0,85.0,6.78,RU,1.623310e+09
3,3,severo-kurilsk,59.7035,30.7861,73.80,81.0,40.0,3.74,RU,1.623310e+09
4,4,nikolskoye,20.5169,-13.0499,86.20,0.0,36.0,4.81,MR,1.623310e+09


In [3]:
city_data.count()

Unnamed: 0    545
City          545
Lat           545
Lng           545
Max Temp      545
Cloudiness    545
Humidity      545
Wind Speed    545
Country       542
Date          545
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#configure maps
gmaps.configure(api_key=g_key)

In [5]:
#Use the Lat and Lng as location
locations = city_data[["Lat", "Lng"]]

#Humidity as the weight
humidity = city_data["Humidity"]
weight = city_data["Humidity"]

In [6]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

## Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
# find ideal cities that fit weather conditions
ideal_city = city_data.loc[(city_data['Max Temp'] < 80) & (city_data['Max Temp']>70)\
                         & (city_data['Wind Speed'] < 10)\
                         & (city_data['Cloudiness'] == 0)] 
#store as a new variable
ideal_city = ideal_city.copy()

#clean up DataSet by remooving NAN
clean_ideal = ideal_city.dropna()

clean_ideal

,Unnamed: 0,City,Lat,Lng,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Date
19,19,santa vitoria do palmar,49.5776,111.9753,78.04,0.0,20.0,4.90,RU,1.623310e+09
20,20,chuy,46.4333,26.4000,71.76,0.0,56.0,5.30,RO,1.623310e+09
58,58,macaboboni,48.8264,58.1442,78.13,0.0,36.0,6.38,KZ,1.623310e+09
255,255,tura,64.4743,40.8524,75.25,0.0,33.0,6.38,RU,1.623310e+09
292,292,san juan,32.1167,20.0667,71.26,0.0,74.0,9.95,LY,1.623310e+09
359,359,chernyshevskiy,33.7333,135.9833,79.79,0.0,67.0,6.49,JP,1.623310e+09
362,362,zarubino,-5.7950,-35.2094,72.25,0.0,94.0,8.90,BR,1.623310e+09
426,426,alice springs,47.4072,32.4387,70.59,0.0,64.0,5.97,UA,1.623310e+09
427,427,tiarei,31.6315,-8.0083,78.87,0.0,38.0,4.61,MA,1.623310e+09
429,429,naberera,42.6667,9.3333,79.93,0.0,53.0,2.59,FR,1.623310e+09


# # Hotel Map¶

* Set parameters to search for hotels with 5000 meters.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
#Store into variable named hotel_df
hotel_df = clean_ideal

hotel_df = hotel_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""


In [24]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [25]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
       
    print("--------------------Ending Search")

Retrieving Results for Index 19: santa vitoria do palmar.
Closest hotel in santa vitoria do palmar is СберБанк.
--------------------Ending Search
Retrieving Results for Index 20: chuy.
Closest hotel in chuy is Pensiunea Sophia.
--------------------Ending Search
Retrieving Results for Index 58: macaboboni.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 255: tura.
Closest hotel in tura is Turkompleks Malyye Karely.
--------------------Ending Search
Retrieving Results for Index 292: san juan.
Closest hotel in san juan is Tibesti Hotel.
--------------------Ending Search
Retrieving Results for Index 359: chernyshevskiy.
Closest hotel in chernyshevskiy is Shingu UI Hotel.
--------------------Ending Search
Retrieving Results for Index 362: zarubino.
Closest hotel in zarubino is eSuites VILA DO MAR Natal.
--------------------Ending Search
Retrieving Results for Index 426: alice springs.
Closest hotel in alice springs is Hotelʹ "Ve

In [26]:
#Store into variable named hotel_df
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Lat,Lng,Hotel Name
19,santa vitoria do palmar,RU,49.5776,111.9753,СберБанк
20,chuy,RO,46.4333,26.4000,Pensiunea Sophia
58,macaboboni,KZ,48.8264,58.1442,
255,tura,RU,64.4743,40.8524,Turkompleks Malyye Karely
292,san juan,LY,32.1167,20.0667,Tibesti Hotel
359,chernyshevskiy,JP,33.7333,135.9833,Shingu UI Hotel
362,zarubino,BR,-5.7950,-35.2094,eSuites VILA DO MAR Natal
426,alice springs,UA,47.4072,32.4387,"Hotelʹ ""Veles"""
427,tiarei,MA,31.6315,-8.0083,La Mamounia
429,naberera,FR,42.6667,9.3333,Hôtel Dolce Notte


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
19,49.5776,111.9753
20,46.4333,26.4000
58,48.8264,58.1442
255,64.4743,40.8524
292,32.1167,20.0667
359,33.7333,135.9833
362,-5.7950,-35.2094
426,47.4072,32.4387
427,31.6315,-8.0083
429,42.6667,9.3333


In [28]:
#Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at a readable size
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
markers = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…